In [1]:

from transformers import LlamaTokenizerFast, MistralForSequenceClassification, AutoConfig

hf_repo = 'h2oai/h2o-danube2-1.8b-base'

max_seq_length = 2048

tokenizer = LlamaTokenizerFast.from_pretrained(hf_repo)

# configure tokenizer to actually use sequence tokens to delimit premise and hypothesis
tokenizer.add_eos_token = True
tokenizer.add_bos_token = True

# config = AutoConfig.from_pretrained(f'{run_of_interest}/config_checkpoint')

model = MistralForSequenceClassification.from_pretrained(hf_repo, num_labels=3)

# Experimenting with freezing earlier parameters
if tokenizer.pad_token is None:
    num_added_tokens = tokenizer.add_special_tokens({'pad_token': '<|pad_token|>'})
    assert num_added_tokens == 1
    model.resize_token_embeddings(len(tokenizer))
    # model.config.pad_token = tokenizer.pad_token
    model.config.pad_token_id = tokenizer.pad_token_id

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at h2oai/h2o-danube2-1.8b-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
print(model)

MistralForSequenceClassification(
  (model): MistralModel(
    (embed_tokens): Embedding(32001, 2560)
    (layers): ModuleList(
      (0-23): 24 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=640, bias=False)
          (v_proj): Linear(in_features=2560, out_features=640, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): Mistra

In [3]:
# for param in model.parameters():
#     print(param)
#     exit()
    # param.requires_grad = False

In [17]:
len(model.model.layers)

24

In [14]:

# Fine-tune only the final normalization layer
last_block = model.model.layers[-1]
print(last_block)

MistralDecoderLayer(
  (self_attn): MistralSdpaAttention(
    (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
    (k_proj): Linear(in_features=2560, out_features=640, bias=False)
    (v_proj): Linear(in_features=2560, out_features=640, bias=False)
    (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
    (rotary_emb): MistralRotaryEmbedding()
  )
  (mlp): MistralMLP(
    (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
    (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
    (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): MistralRMSNorm()
  (post_attention_layernorm): MistralRMSNorm()
)


In [16]:

print(len(list(last_block.parameters())[0]))

2560


In [5]:

final_norm = model.model.norm
print(final_norm)

MistralRMSNorm()


In [13]:

print(len(list(final_norm.parameters())[0]))

2560
